# Strategy Analysis

**Run ID**: `20250624_150142`  
**Generated**: 2025-06-24 15:41:12  
**Config**: `unnamed`  

## Quick Navigation
1. [Setup](#setup)
2. [Load Data](#load-data)
3. [Signal Analysis](#signal-analysis)
4. [Performance Metrics](#performance-metrics)
5. [Visualizations](#visualizations)
6. [Export Results](#export-results)

In [ ]:
# Auto-generated setup
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)

# Set paths
results_path = Path('/Users/daws/ADMF-PC/config/bollinger/results/20250624_150142')
print(f'Analyzing results in: {results_path}')
print(f'Run ID: {results_path.name}')

# Initialize DuckDB
con = duckdb.connect()

# Helper functions
def format_number(x):
    if abs(x) >= 1e6:
        return f'{x/1e6:.1f}M'
    elif abs(x) >= 1e3:
        return f'{x/1e3:.1f}K'
    else:
        return f'{x:.2f}'

## Strategy Analysis

In [ ]:
# Generic strategy analysis
print('Available files:')
for f in results_path.rglob('*.parquet'):
    print(f'  {f.relative_to(results_path)}')

# Basic signal statistics
signal_count = con.execute(f"""
    SELECT 
        COUNT(*) as total_signals,
        COUNT(DISTINCT strat) as unique_strategies,
        COUNT(DISTINCT DATE(ts)) as trading_days
    FROM read_parquet('{results_path}/traces/**/*.parquet')
    WHERE val != 0
""").df()

print(f'\nSignal Statistics:')
print(signal_count)

## Performance Metrics <a name='performance-metrics'></a>

In [ ]:
# Load market data for performance calculation
# Try to find market data in standard locations
market_data_paths = [
    Path('data/SPY_5m.parquet'),
    Path('../data/SPY_5m.parquet'),
    Path('../../data/SPY_5m.parquet'),
    Path('../../../data/SPY_5m.parquet'),
    Path('../../../../data/SPY_5m.parquet'),
]

market_data = None
for path in market_data_paths:
    if path.exists():
        market_data = pd.read_parquet(path)
        print(f'Loaded market data from: {path}')
        print(f'Market data shape: {market_data.shape}')
        break

if market_data is None:
    print('WARNING: Could not find market data file. Performance calculation will be skipped.')
    print('Searched in:', market_data_paths)

In [ ]:
def calculate_performance(strategy_hash, signals_df, market_data):
    """Calculate performance metrics for a strategy"""
    # Merge with market data
    df = market_data.merge(
        signals_df[['ts', 'val']], 
        left_on='timestamp', 
        right_on='ts', 
        how='left'
    )
    
    # Forward fill signals
    df['signal'] = df['val'].fillna(method='ffill').fillna(0)
    
    # Calculate returns
    df['returns'] = df['close'].pct_change()
    df['strategy_returns'] = df['returns'] * df['signal'].shift(1)
    df['cum_returns'] = (1 + df['strategy_returns']).cumprod()
    
    # Calculate metrics
    total_return = df['cum_returns'].iloc[-1] - 1
    
    # Annualized Sharpe (assuming 5-minute bars, 78 per day, 252 trading days)
    if df['strategy_returns'].std() > 0:
        sharpe = df['strategy_returns'].mean() / df['strategy_returns'].std() * np.sqrt(252 * 78)
    else:
        sharpe = 0
    
    # Max drawdown
    cummax = df['cum_returns'].expanding().max()
    drawdown = (df['cum_returns'] / cummax - 1)
    max_dd = drawdown.min()
    
    # Win rate
    winning_trades = (df['strategy_returns'] > 0).sum()
    losing_trades = (df['strategy_returns'] < 0).sum()
    total_trades = winning_trades + losing_trades
    win_rate = winning_trades / total_trades if total_trades > 0 else 0
    
    return {
        'strategy_hash': strategy_hash,
        'total_return': total_return,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_dd,
        'win_rate': win_rate,
        'total_trades': total_trades,
        'df': df  # Return full dataframe for plotting
    }

In [ ]:
# Calculate performance for strategies
performance_results = []

if market_data is not None and strategy_index is not None:
    # Limit to top N strategies to avoid long computation
    strategies_to_analyze = strategy_index.head(20)
    
    print(f'Calculating performance for {len(strategies_to_analyze)} strategies...')
    
    for idx, row in strategies_to_analyze.iterrows():
        # Load signals for this strategy
        trace_path = results_path / row['trace_path']
        
        if trace_path.exists():
            signals = pd.read_parquet(trace_path)
            signals['ts'] = pd.to_datetime(signals['ts'])
            
            perf = calculate_performance(row['strategy_hash'], signals, market_data)
            
            # Add strategy metadata
            perf['strategy_type'] = row.get('strategy_type', 'unknown')
            perf['period'] = row.get('param_period')
            perf['std_dev'] = row.get('param_std_dev')
            
            performance_results.append(perf)
    
    if performance_results:
        performance_df = pd.DataFrame(performance_results)
        print(f'\nCalculated performance for {len(performance_df)} strategies')
        
        # Show top performers
        print('\nTop 10 by Sharpe Ratio:')
        cols_to_show = ['strategy_type', 'period', 'std_dev', 'sharpe_ratio', 'total_return', 'max_drawdown', 'win_rate']
        cols_to_show = [col for col in cols_to_show if col in performance_df.columns]
        top_performers = performance_df.nlargest(10, 'sharpe_ratio')[cols_to_show].round(3)
        print(top_performers)
    else:
        print('No performance results calculated')
        performance_df = pd.DataFrame()
else:
    print('Skipping performance calculation - missing market data or strategy index')
    performance_df = pd.DataFrame()

## Pattern Discovery

In [ ]:
# Discover patterns in successful strategies
if len(performance_df) > 0:
    # Find common characteristics in top performers
    top_n = min(10, len(performance_df))
    top_performers = performance_df.nlargest(top_n, 'sharpe_ratio')
    
    print(f'Analyzing top {top_n} strategies...')
    
    # Parameter patterns
    if 'period' in top_performers.columns:
        print(f'\nPeriod distribution in top performers:')
        print(top_performers['period'].value_counts().sort_index())
    
    if 'std_dev' in top_performers.columns:
        print(f'\nStd Dev distribution in top performers:')
        print(top_performers['std_dev'].value_counts().sort_index())
    
    # Save pattern for future use
    if len(top_performers) > 0:
        best = top_performers.iloc[0]
        pattern = {
            'name': f'{best.get("strategy_type", "unknown")}_high_sharpe',
            'discovered_at': datetime.now().isoformat(),
            'run_id': results_path.name,
            'performance': {
                'sharpe_ratio': float(best['sharpe_ratio']),
                'total_return': float(best['total_return']),
                'max_drawdown': float(best['max_drawdown'])
            },
            'parameters': {}
        }
        
        # Add available parameters
        for param in ['period', 'std_dev', 'fast_period', 'slow_period']:
            if param in best:
                pattern['parameters'][param] = best[param]
        
        print(f'\nDiscovered pattern: {pattern["name"]}')
        print(f'Parameters: {pattern["parameters"]}')

## Visualizations <a name='visualizations'></a>

In [ ]:
# Plot best strategy equity curve
if len(performance_df) > 0 and 'df' in performance_df.columns:
    best_idx = performance_df['sharpe_ratio'].idxmax()
    best_strategy = performance_df.loc[best_idx]
    
    plt.figure(figsize=(15, 8))
    
    # Equity curve
    plt.subplot(2, 1, 1)
    df = best_strategy['df']
    plt.plot(df.index, df['cum_returns'], label='Strategy', linewidth=2)
    
    # Add buy & hold for comparison
    df['bh_returns'] = (1 + df['returns']).cumprod()
    plt.plot(df.index, df['bh_returns'], label='Buy & Hold', alpha=0.7)
    
    plt.title(f'Best Strategy Performance (Sharpe: {best_strategy["sharpe_ratio"]:.2f})')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Drawdown
    plt.subplot(2, 1, 2)
    cummax = df['cum_returns'].expanding().max()
    drawdown = (df['cum_returns'] / cummax - 1)
    plt.fill_between(df.index, drawdown, 0, alpha=0.3, color='red')
    plt.ylabel('Drawdown')
    plt.xlabel('Time')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print strategy details
    print(f'\nBest Strategy Details:')
    print(f'Type: {best_strategy.get("strategy_type", "unknown")}')
    if 'period' in best_strategy:
        print(f'Period: {best_strategy["period"]}')
    if 'std_dev' in best_strategy:
        print(f'Std Dev: {best_strategy["std_dev"]}')

In [ ]:
# Parameter sensitivity analysis
if len(performance_df) > 0 and 'period' in performance_df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Sharpe by period
    if 'period' in performance_df.columns:
        period_stats = performance_df.groupby('period')['sharpe_ratio'].agg(['mean', 'std'])
        period_stats['mean'].plot(ax=axes[0, 0], marker='o')
        axes[0, 0].fill_between(period_stats.index, 
                               period_stats['mean'] - period_stats['std'],
                               period_stats['mean'] + period_stats['std'],
                               alpha=0.3)
        axes[0, 0].set_title('Sharpe Ratio by Period')
        axes[0, 0].set_xlabel('Period')
        axes[0, 0].grid(True)
    
    # Sharpe by std_dev
    if 'std_dev' in performance_df.columns:
        std_stats = performance_df.groupby('std_dev')['sharpe_ratio'].agg(['mean', 'std'])
        std_stats['mean'].plot(ax=axes[0, 1], marker='o')
        axes[0, 1].set_title('Sharpe Ratio by Std Dev')
        axes[0, 1].set_xlabel('Standard Deviation')
        axes[0, 1].grid(True)
    
    # Return vs Risk scatter
    axes[1, 0].scatter(performance_df['total_return'], performance_df['sharpe_ratio'], alpha=0.6)
    axes[1, 0].set_xlabel('Total Return')
    axes[1, 0].set_ylabel('Sharpe Ratio')
    axes[1, 0].set_title('Return vs Risk-Adjusted Return')
    axes[1, 0].grid(True)
    
    # Drawdown vs Sharpe
    axes[1, 1].scatter(performance_df['max_drawdown'], performance_df['sharpe_ratio'], alpha=0.6)
    axes[1, 1].set_xlabel('Max Drawdown')
    axes[1, 1].set_ylabel('Sharpe Ratio')
    axes[1, 1].set_title('Drawdown vs Sharpe')
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()